In [1]:
from fastai.vision import *

classes = ['facade', 'interior']

In [2]:
# TODO properties
model = load_learner('train/', 'fastai.pkl')

In [19]:
# TODO properties + create path_new_images
path_images = "images"
path_new_images = "images_predicted"

for i in os.listdir(path_images):
    if i.endswith('.jpg'): # TODO png?
        img = open_image(os.path.join(path_images, i))
        img_title, img_extension = os.path.splitext(i)
        pred_class, pred_idx, outputs = model.predict(img)
        prediction = (classes[int(pred_class)], round(max(outputs.numpy()) * 100))
        
        new_img_title = img_title + "_" + prediction[0] + "_" + str(prediction[1]) + img_extension
        new_img_path = os.path.join(path_new_images, new_img_title)
        img.save(new_img_path)

1_4_66885443_interior_100.jpg
images_predicted/1_4_66885443_interior_100.jpg
Image (3, 853, 1280)
1_4_66886918_facade_100.jpg
images_predicted/1_4_66886918_facade_100.jpg
Image (3, 853, 1280)
